In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from collections import defaultdict, Counter
import numpy as np

In [2]:
df = pd.read_csv('lr3/Shakespeare_data.csv')

In [3]:
df.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [4]:
filtered_data = df[~df['PlayerLine'].str.contains("ACT|SCENE", na=False)].copy()
filtered_data['PlayerLine'] = filtered_data['PlayerLine'].str.lower()
filtered_data['PlayerLine'] = filtered_data['PlayerLine'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [5]:
train_data, test_data = train_test_split(filtered_data, test_size=0.2, random_state=42)

In [6]:
from collections import defaultdict, Counter
import numpy as np

class NgramModel:
    def __init__(self, n=3, smoothing=1):
        """
        Ініціалізація N-грамної моделі.
        :param n: Розмір N-грами.
        :param smoothing: Значення для Лапласового згладжування.
        """
        self.n = n
        self.smoothing = smoothing
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = Counter()

    def train(self, text):
        """
        Навчання N-грамної моделі на заданому тексті.
        :param text: Список токенізованих речень.
        """
        for sentence in text:
            tokens = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i+self.n])
                context = ngram[:-1]
                token = ngram[-1]
                self.ngram_counts[context][token] += 1
                self.context_counts[context] += 1

    def ngram_probability(self, context, token):
        """
        Обчислення ймовірності N-грами з Лапласовим згладжуванням.
        :param context: Контекст N-грами.
        :param token: Токен N-грами.
        :return: Ймовірність токена в даному контексті.
        """
        count = self.ngram_counts[context][token]
        total_count = self.context_counts[context]
        vocab_size = len(self.ngram_counts)
        return (count + self.smoothing) / (total_count + self.smoothing * vocab_size)

    def generate_text(self, length=10, k=5):
        """
        Генерація тексту на основі моделі з використанням top-k стратегії.
        :param length: Довжина тексту, який треба згенерувати.
        :param k: Кількість варіантів для вибору наступного токена.
        :return: Згенерований текст.
        """
        context = ('<s>',) * (self.n - 1)
        result = list(context)
        
        for _ in range(length):
            candidates = self.ngram_counts[context]
            if not candidates:
                break
            top_k = Counter({token: self.ngram_probability(context, token) for token in candidates}).most_common(k)
            tokens, probabilities = zip(*top_k)
            next_token = np.random.choice(tokens, p=np.array(probabilities) / sum(probabilities))
            result.append(next_token)
            context = (*context[1:], next_token)
            if next_token == '</s>':
                break
        
        return ' '.join(result).replace('<s>', '').replace('</s>', '').strip()


ngram_model = NgramModel(n=3)
sample_text = [
    "to be or not to be".split(),
    "that is the question".split(),
    "whether tis nobler in the mind to suffer".split()
]
ngram_model.train(sample_text)
generated_text = ngram_model.generate_text(length=10, k=5)
generated_text


'that is the question'

In [7]:
class NgramModelWithPerplexity(NgramModel):
    def perplexity(self, test_text):
        """
        Обчислення перплексії на тестовому тексті.
        :param test_text: Список токенізованих речень для тестування.
        :return: Значення перплексії.
        """
        log_prob_sum = 0
        word_count = 0
        
        for sentence in test_text:
            tokens = ['<s>'] * (self.n - 1) + sentence + ['</s>']
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i+self.n-1])
                token = tokens[i+self.n-1]
                prob = self.ngram_probability(ngram, token)
                log_prob_sum += np.log(prob) if prob > 0 else float('-inf')
                word_count += 1
                
        perplexity = np.exp(-log_prob_sum / word_count) if word_count > 0 else float('inf')
        return perplexity

train_text = [line.split() for line in train_data['PlayerLine'].dropna()]
test_text = [line.split() for line in test_data['PlayerLine'].dropna()]
def train_and_evaluate_ngram_models(train_text, test_text, n_values=[3, 5, 10]):
    results = {}
    
    for n in n_values:
        model = NgramModelWithPerplexity(n=n)
        model.train(train_text)
        perplexity = model.perplexity(test_text)
        results[n] = perplexity
    
    return results
model_results = train_and_evaluate_ngram_models(train_text, test_text, n_values=[3, 5, 10])
model_results


{3: 77227.74040444875, 5: 223365.61041386478, 10: 228597.952037149}

In [8]:
class NgramModelWithTopK(NgramModelWithPerplexity):
    def generate_text_with_top_k(self, length=10, k=5):
        """
        Генерація тексту з використанням top-k стратегії.
        :param length: Довжина тексту, який треба згенерувати.
        :param k: Кількість топ-варіантів для вибору наступного слова.
        :return: Згенерований текст.
        """
        context = ('<s>',) * (self.n - 1)
        result = list(context)

        for _ in range(length):
            candidates = self.ngram_counts[context]
            if not candidates:
                break
            top_k = Counter({token: self.ngram_probability(context, token) for token in candidates}).most_common(k)
            tokens, probabilities = zip(*top_k)

            probabilities = np.array(probabilities) / sum(probabilities)
            next_token = np.random.choice(tokens, p=probabilities)

            result.append(next_token)
            context = (*context[1:], next_token)

            if next_token == '</s>':
                break

        return ' '.join(result).replace('<s>', '').replace('</s>', '').strip()

ngram_model_top_k = NgramModelWithTopK(n=3)
ngram_model_top_k.train(train_text)

generated_text_top_k = ngram_model_top_k.generate_text_with_top_k(length=20, k=5)
generated_text_top_k


'i have done'

In [9]:
import gradio as gr
import numpy as np

def generate_text(n, strategy, k, start_text, max_length):
    model = NgramModelWithTopK(n=n)
    model.train(train_text)  
    
    start_tokens = start_text.lower().split()
    length = max_length
    
    if strategy == 'top-k':
        generated_text = model.generate_text_with_top_k(length=length, k=k)
    else:
        generated_text = model.generate_text(length=length)  # Greedy

    tokens = generated_text.split()
    entropies = []
    for i in range(len(tokens) - n + 1):
        context = tuple(tokens[i:i+n-1])
        token = tokens[i+n-1]
        prob = model.ngram_probability(context, token)
        entropies.append(-np.log(prob) if prob > 0 else float('inf'))
    
    avg_entropy = np.mean(entropies) if entropies else float('inf')
    return generated_text, avg_entropy


with gr.Blocks() as demo:
    gr.Markdown("# Генерація тексту за допомогою N-грамної моделі")
    n = gr.Slider(3, 10, step=1, label="Виберіть N для моделі", value=3)
    strategy = gr.Radio(["greedy", "top-k"], label="Виберіть стратегію генерації")
    k = gr.Slider(1, 20, step=1, label="Виберіть значення k для top-k стратегії", value=5)
    start_text = gr.Textbox(label="Початковий текст")
    max_length = gr.Slider(5, 50, step=1, label="Максимальна довжина згенерованого тексту", value=20)
    
    generate_button = gr.Button("Згенерувати текст")
    output_text = gr.Textbox(label="Згенерований текст")
    entropy = gr.Number(label="Ентропія згенерованого тексту")
    
    generate_button.click(generate_text, inputs=[n, strategy, k, start_text, max_length], outputs=[output_text, entropy])

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


to be or

wherefore art thou

all the world's a stage

love looks not

In [10]:
import nltk
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import random

In [11]:
train_text_tokens = [list(line) for line in train_text]
test_text_tokens = [list(line) for line in test_text]

In [16]:
import gradio as gr
import nltk
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import numpy as np

In [17]:
def train_ngram_model(n, train_text_tokens):
    train_data, padded_vocab = padded_everygram_pipeline(n, train_text_tokens)
    model = Laplace(n)
    model.fit(train_data, padded_vocab)
    return model

In [18]:
def generate_text(model, n, start_text, max_length=15, strategy="top-k", k=5):
    context = start_text.split()
    generated = context.copy()
    entropies = []
    
    for _ in range(max_length):
        context_ngram = tuple(context[-(n-1):]) if len(context) >= n - 1 else tuple(context)
        
        if strategy == "greedy":
            next_word = model.generate(text_seed=context_ngram)
        elif strategy == "top-k":
            candidates = [(word, model.score(word, context_ngram)) for word in model.vocab]
            candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:k]
            if not candidates:
                break
            words, probabilities = zip(*candidates)
            probabilities = np.array(probabilities) / sum(probabilities)
            next_word = np.random.choice(words, p=probabilities)
        
        if next_word == '</s>':
            break
        generated.append(next_word)
        context.append(next_word)
        
        prob = model.score(next_word, context_ngram)
        if prob > 0:
            entropies.append(-np.log(prob))
    
    avg_entropy = np.mean(entropies) if entropies else float('inf')
    return ' '.join(generated), avg_entropy

train_text_tokens = [list(line.split()) for line in ["to be or not to be", "that is the question"]]


In [19]:
def gradio_app(start_text, n, max_length, strategy, k):
    model = train_ngram_model(n, train_text_tokens)
    generated_text, avg_entropy = generate_text(model, n, start_text, max_length, strategy, k)
    return generated_text, avg_entropy

with gr.Blocks() as demo:
    gr.Markdown("# Генерація тексту за допомогою N-грамної моделі")
    start_text = gr.Textbox(label="Початковий текст")
    n = gr.Slider(3, 10, step=1, label="Виберіть N для моделі", value=3)
    max_length = gr.Slider(5, 50, step=1, label="Максимальна довжина згенерованого тексту", value=20)
    strategy = gr.Radio(["greedy", "top-k"], label="Виберіть стратегію генерації")
    k = gr.Slider(1, 20, step=1, label="Виберіть значення k для top-k стратегії", value=5)
    
    generate_button = gr.Button("Згенерувати текст")
    output_text = gr.Textbox(label="Згенерований текст")
    entropy = gr.Number(label="Ентропія згенерованого тексту")
    
    generate_button.click(gradio_app, inputs=[start_text, n, max_length, strategy, k], outputs=[output_text, entropy])

demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# Варіант 3 Тільки текст

In [32]:
import re
import nltk

import random
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from sklearn.model_selection import train_test_split

import numpy as np
import gradio as gr

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Oleg_PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [14]:
file_path = 'lr3/alllines.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [15]:
filtered_lines = []
for line in lines:
    line = line.strip()
    if not (line.startswith("ACT") or line.startswith("SCENE") or "Enter" in line):
        filtered_lines.append(line)

In [16]:
text = ' '.join(filtered_lines)

In [17]:
sentences = nltk.sent_tokenize(text)

In [18]:
processed_sentences = []
for sentence in sentences:
    words = nltk.word_tokenize(sentence.lower())
    words = [re.sub(r'[^\w\s]', '', word) for word in words if word.isalpha()] 
    if words:  
        processed_sentences.append(words)

In [19]:
for i in range(5):
    print(processed_sentences[i])

['act', 'i', 'scene', 'london']
['the', 'palace']
['so', 'shaken', 'as', 'we', 'are', 'so', 'wan', 'with', 'care', 'find', 'we', 'a', 'time', 'for', 'frighted', 'peace', 'to', 'pant', 'and', 'breathe', 'accents', 'of', 'new', 'broils', 'to', 'be', 'commenced', 'in', 'strands', 'afar', 'remote']
['no', 'more', 'the', 'thirsty', 'entrance', 'of', 'this', 'soil', 'shall', 'daub', 'her', 'lips', 'with', 'her', 'own', 'children', 'blood', 'nor', 'more', 'shall', 'trenching', 'war', 'channel', 'her', 'fields', 'nor', 'bruise', 'her', 'flowerets', 'with', 'the', 'armed', 'hoofs', 'of', 'hostile', 'paces', 'those', 'opposed', 'eyes', 'which', 'like', 'the', 'meteors', 'of', 'a', 'troubled', 'heaven', 'all', 'of', 'one', 'nature', 'of', 'one', 'substance', 'bred', 'did', 'lately', 'meet', 'in', 'the', 'intestine', 'shock', 'and', 'furious', 'close', 'of', 'civil', 'butchery', 'shall', 'now', 'in', 'mutual', 'ranks', 'march', 'all', 'one', 'way', 'and', 'be', 'no', 'more', 'opposed', 'against', 

In [20]:
train_sentences, test_sentences = train_test_split(processed_sentences, test_size=0.2, random_state=42)


In [24]:
def train_ngram_model(n, train_sentences):
    train_data, vocab = padded_everygram_pipeline(n, train_sentences)
    model = Laplace(n)
    model.fit(train_data, vocab)
    return model

In [25]:
models = {}
for n in [3, 5, 10]:
    print(f"Навчання {n}-грамної моделі...")
    models[n] = train_ngram_model(n, train_sentences)
    print(f"{n}-грамна модель навчена.")

Навчання 3-грамної моделі...
3-грамна модель навчена.
Навчання 5-грамної моделі...
5-грамна модель навчена.
Навчання 10-грамної моделі...
10-грамна модель навчена.


In [27]:
def evaluate_model(model, test_sentences, n):
    total_log_prob = 0
    word_count = 0
    entropies = []
    
    for sentence in test_sentences:
        # Генерація n-грам для поточного речення
        ngrams = list(nltk.ngrams(['<s>'] * (n - 1) + sentence + ['</s>'], n))
        
        for ngram in ngrams:
            context = ngram[:-1]
            word = ngram[-1]
            prob = model.score(word, context)
            log_prob = np.log(prob) if prob > 0 else float('-inf')
            total_log_prob += log_prob
            entropies.append(-log_prob if prob > 0 else 0)
            word_count += 1
    
    # Обчислення перплексії
    perplexity = np.exp(-total_log_prob / word_count) if word_count > 0 else float('inf')
    # Середня ентропія
    avg_entropy = np.mean(entropies) if entropies else float('inf')
    return perplexity, avg_entropy

In [28]:
evaluation_results = {}
for n, model in models.items():
    print(f"Оцінка {n}-грамної моделі...")
    perplexity, avg_entropy = evaluate_model(model, test_sentences, n)
    evaluation_results[n] = {'perplexity': perplexity, 'entropy': avg_entropy}
    print(f"{n}-грамна модель: Перплексія = {perplexity}, Ентропія = {avg_entropy}")

Оцінка 3-грамної моделі...
3-грамна модель: Перплексія = 9596.57301883139, Ентропія = 9.169161336516845
Оцінка 5-грамної моделі...
5-грамна модель: Перплексія = 13225.561769508507, Ентропія = 9.489906733652935
Оцінка 10-грамної моделі...
10-грамна модель: Перплексія = 13301.64022190348, Ентропія = 9.495642631543033


In [30]:
def generate_text(model, n, start_text, max_length=15, strategy="top-k", k=5):
    context = start_text.split()
    generated_text = context.copy()
    entropies = []
    
    for _ in range(max_length):
        context_ngram = tuple(context[-(n-1):]) if len(context) >= n - 1 else tuple(context)
        
        if strategy == "greedy":
            next_word = model.generate(text_seed=context_ngram)
        elif strategy == "top-k":
            candidates = [(word, model.score(word, context_ngram)) for word in model.vocab]
            candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:k]
            if not candidates:
                break
            words, probabilities = zip(*candidates)
            probabilities = np.array(probabilities) / sum(probabilities)
            next_word = np.random.choice(words, p=probabilities)
        
        if next_word == '</s>':
            break
        generated_text.append(next_word)
        context.append(next_word)
        
        
        prob = model.score(next_word, context_ngram)
        if prob > 0:
            entropies.append(-np.log(prob))
    
    avg_entropy = np.mean(entropies) if entropies else float('inf')
    return ' '.join(generated_text), avg_entropy

In [33]:
# Створення Gradio інтерфейсу
def gradio_app(start_text, n, max_length, strategy, k):
    model = train_ngram_model(n, train_sentences)
    generated_text, avg_entropy = generate_text(model, n, start_text, max_length, strategy, k)
    return generated_text, avg_entropy

# Налаштування інтерфейсу
with gr.Blocks() as demo:
    gr.Markdown("# Генерація тексту за допомогою N-грамної моделі")
    start_text = gr.Textbox(label="Початковий текст")
    n = gr.Slider(3, 10, step=1, label="Виберіть N для моделі", value=3)
    max_length = gr.Slider(5, 50, step=1, label="Максимальна довжина згенерованого тексту", value=20)
    strategy = gr.Radio(["greedy", "top-k"], label="Виберіть стратегію генерації")
    k = gr.Slider(1, 20, step=1, label="Виберіть значення k для top-k стратегії", value=5)
    
    generate_button = gr.Button("Згенерувати текст")
    output_text = gr.Textbox(label="Згенерований текст")
    entropy = gr.Number(label="Ентропія згенерованого тексту")
    
    generate_button.click(gradio_app, inputs=[start_text, n, max_length, strategy, k], outputs=[output_text, entropy])

# Запуск Gradio демо
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
